In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, agent_state_dim, target_feature_dim, embed_dim=128, num_heads=4, distance_mode='log'):
        super(Attention, self).__init__()
        self.embed_dim = embed_dim
        self.distance_mode = distance_mode
        
        self.agent_encoder = nn.Sequential(
            nn.Linear(agent_state_dim, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, embed_dim)
        )

        self.target_encoder = nn.Sequential(
            nn.Linear(target_feature_dim, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, embed_dim)
        )

        self.agent_self_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.norm_agent = nn.LayerNorm(embed_dim)
        
        # Input cho lớp quyết định: Agent feat + Target feat + Distance
        self.assignment_attention = nn.Sequential(
            nn.Linear(embed_dim * 2 + 1, 256), 
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1) 
        )

    def forward(self, agent_states, global_targets, assignment_mode='gumbel', temperature=1.0):
        batch_size, num_agents, _ = agent_states.size()
        _, num_targets, _ = global_targets.size()

        agent_emb = self.agent_encoder(agent_states)      # [B, N, Emb]
        target_emb = self.target_encoder(global_targets)  # [B, M, Emb]

        # Self-attention giữa các agents
        agent_context, _ = self.agent_self_attn(query=agent_emb, key=agent_emb, value=agent_emb)
        agent_features = self.norm_agent(agent_emb + agent_context)
        
        # Tính khoảng cách
        a_pos = agent_states[:, :, :2].unsqueeze(2) # [B, N, 1, 2]
        t_pos = global_targets[:, :, :2].unsqueeze(1) # [B, 1, M, 2]
        dist_vec = a_pos - t_pos 
        dist = torch.norm(dist_vec, dim=-1, keepdim=True) # [B, N, M, 1]
        
        if self.distance_mode == 'log':
            dist_scalar = torch.log(dist + 1e-6)
        else:
            dist_scalar = dist

        # Expand dimensions để ghép nối (Concatenate)
        agent_features_exp = agent_features.unsqueeze(2).expand(-1, -1, num_targets, -1)
        target_emb_exp = target_emb.unsqueeze(1).expand(-1, num_agents, -1, -1)

        combined = torch.cat([agent_features_exp, target_emb_exp, dist_scalar], dim=-1)

        logits = self.assignment_attention(combined).squeeze(-1) # [B, N, M]
        
        # --- SỬA LỖI LOGIC TẠI ĐÂY ---
        if assignment_mode == 'gumbel':
            if self.training:
                # Hard=True trả về one-hot nhưng có gradient cho backprop
                weights = F.gumbel_softmax(logits, tau=temperature, hard=True, dim=-1)
            else:
                indices = torch.argmax(logits, dim=-1)
                weights = F.one_hot(indices, num_classes=num_targets).float()
        else:
            weights = torch.sigmoid(logits) # Soft probability

        return weights # Đảm bảo dòng này nằm ngoài cùng, không thuộc if/else nào

In [14]:
import torch
import numpy as np
import matplotlib.pyplot as plt

class SimulationEnv:
    def __init__(self, num_agents=4, num_targets=5, area_size=100.0, device='cpu'):
        self.num_agents = num_agents
        self.num_targets = num_targets
        self.area_size = area_size
        self.device = device
        self.max_speed_agent = 2.0
        self.max_speed_target = 1.0
        
        # Ngưỡng bao phủ (Agent cần đến gần Target bao nhiêu mét thì tính là cover)
        self.coverage_radius = 10.0 
        
        self.reset()

    def reset(self):
        # Khởi tạo vị trí ngẫu nhiên [x, y]
        self.agent_pos = torch.rand(1, self.num_agents, 2).to(self.device) * self.area_size
        self.target_pos = torch.rand(1, self.num_targets, 2).to(self.device) * self.area_size
        
        # Vận tốc target ngẫu nhiên
        self.target_vel = (torch.rand(1, self.num_targets, 2).to(self.device) - 0.5) * 2 * self.max_speed_target
        
        # Dummy features cho camera (pan, tilt, zoom) để khớp input model
        self.agent_ptz = torch.zeros(1, self.num_agents, 3).to(self.device)
        
        return self._get_state()

    def _get_state(self):
        # Gom lại thành tensor đúng shape model yêu cầu
        # agent_states: (Batch, N, 5) -> [x, y, pan, tilt, zoom]
        agent_states = torch.cat([self.agent_pos, self.agent_ptz], dim=-1)
        
        # global_targets: (Batch, M, 4) -> [x, y, vx, vy]
        global_targets = torch.cat([self.target_pos, self.target_vel], dim=-1)
        
        return agent_states, global_targets

    def step(self, assignment_weights):
        """
        assignment_weights: (Batch, N, M) - Output từ Gumbel Softmax
        """
        # 1. Di chuyển Targets (Bounce off walls - dội ngược khi gặp tường)
        self.target_pos += self.target_vel
        # Kiểm tra biên và đảo chiều vận tốc nếu chạm tường
        for i in range(2): # x and y
            mask_lower = self.target_pos[:, :, i] < 0
            mask_upper = self.target_pos[:, :, i] > self.area_size
            self.target_vel[:, :, i][mask_lower | mask_upper] *= -1
            self.target_pos[:, :, i] = torch.clamp(self.target_pos[:, :, i], 0, self.area_size)

        # 2. Di chuyển Agents dựa trên Assignment
        # Model chọn target nào (dựa trên trọng số lớn nhất), Agent sẽ đi về hướng đó
        # assignment_weights shape: (1, N, M)
        # target_pos shape: (1, M, 2)
        
        # Tính vị trí mục tiêu mong muốn cho mỗi agent (Weighted Average Position)
        # Vì Gumbel hard=True nên nó sẽ là vị trí của đúng 1 target được chọn
        target_destinations = torch.bmm(assignment_weights, self.target_pos) # (1, N, 2)
        
        # Vector hướng di chuyển
        direction = target_destinations - self.agent_pos
        distance = torch.norm(direction, dim=-1, keepdim=True)
        
        # Chuẩn hóa vector hướng và nhân với tốc độ tối đa
        move_step = (direction / (distance + 1e-6)) * self.max_speed_agent
        
        # Nếu đã ở rất gần target, không cần đi hết tốc độ (tránh rung lắc)
        move_step = torch.where(distance < self.max_speed_agent, direction, move_step)
        
        self.agent_pos += move_step
        self.agent_pos = torch.clamp(self.agent_pos, 0, self.area_size)

        # 3. Tính toán Reward / Stats
        reward, covered_count = self._calculate_reward(self.agent_pos, self.target_pos)
        
        return self._get_state(), reward, covered_count

    def _calculate_reward(self, a_pos, t_pos):
        # Tính khoảng cách giữa tất cả Agent và tất cả Target
        # a_pos: (1, N, 2), t_pos: (1, M, 2)
        dist_matrix = torch.cdist(a_pos, t_pos, p=2) # (1, N, M)
        
        # Kiểm tra xem mỗi target có bị agent nào cover không (dist < R)
        min_dist_per_target, _ = torch.min(dist_matrix, dim=1) # (1, M) - khoảng cách tới agent gần nhất
        
        is_covered = min_dist_per_target < self.coverage_radius
        covered_count = is_covered.sum().item()
        
        # Reward = Tỷ lệ bao phủ
        reward = covered_count / self.num_targets
        
        return reward, covered_count

In [15]:
def compute_custom_loss(weights, agent_pos, target_pos):
    """
    weights: (B, N, M) - Output từ model
    agent_pos: (B, N, 2)
    target_pos: (B, M, 2)
    """
    # 1. Distance Cost: Khuyến khích chọn target ở gần
    dist_matrix = torch.cdist(agent_pos, target_pos) # (B, N, M)
    # Tổng khoảng cách có trọng số (Agent chọn target nào thì tính khoảng cách đó)
    weighted_dist = (weights * dist_matrix).sum() 
    
    # 2. Coverage / Diversity Cost: Khuyến khích bao phủ hết các target
    # Cộng dồn trọng số theo cột (Targets). 
    # Nếu lý tưởng: mỗi target nhận được tổng weight = 1 (hoặc > 0).
    target_coverage = weights.sum(dim=1) # (B, M)
    
    # Chúng ta muốn target_coverage càng đồng đều càng tốt.
    # Phạt các target có coverage = 0.
    # Sử dụng MSE so với mức lý tưởng (ví dụ: N/M hoặc đơn giản là > 0)
    # Ở đây dùng hàm log barrier hoặc MSE loss để ép phân phối đều
    diversity_loss = torch.sum((target_coverage - 1.0) ** 2) 
    
    # Tổng hợp Loss
    # Alpha điều chỉnh sự cân bằng giữa việc "Chọn thằng gần nhất" vs "Chia nhau ra"
    loss = weighted_dist + 10.0 * diversity_loss 
    
    return loss

In [ ]:
# Cấu hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_episodes = 500
steps_per_episode = 50

# Khởi tạo
env = SimulationEnv(num_agents=4, num_targets=5, device=device)
model = Attention(agent_state_dim=5, target_feature_dim=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loss_history = []
reward_history = []

print("Bắt đầu training...")

for episode in range(num_episodes):
    # Reset môi trường, lấy state đầu tiên
    agent_states, global_targets = env.reset()
    
    total_reward = 0
    total_loss = 0
    
    for step in range(steps_per_episode):
        optimizer.zero_grad()
        
        # 1. Forward Pass
        temp = max(0.5, 1.0 - episode / num_episodes)
        
        # Đảm bảo input là Tensor (phòng hờ)
        if isinstance(agent_states, tuple): 
            agent_states = agent_states[0] # Fallback nếu vẫn bị lồng tuple
            
        weights = model(agent_states, global_targets, assignment_mode='gumbel', temperature=temp)
        
        # 2. Tính Loss
        curr_agent_pos = agent_states[:, :, :2]
        curr_target_pos = global_targets[:, :, :2]
        
        loss = compute_custom_loss(weights, curr_agent_pos, curr_target_pos)
        
        # 3. Backward & Update
        loss.backward()
        
        # Gradient Clipping (Mẹo: Giúp train ổn định hơn, tránh gradient bùng nổ)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # 4. Step Environment (QUAN TRỌNG: SỬA LỖI TẠI ĐÂY)
        # env.step trả về: ((agents, targets), reward, count)
        (next_agent_states, next_global_targets), reward, covered_count = env.step(weights.detach())
        
        # Cập nhật state cho vòng lặp sau
        agent_states = next_agent_states
        global_targets = next_global_targets
        
        total_reward += reward
        total_loss += loss.item()

    # Logging
    avg_reward = total_reward / steps_per_episode
    loss_history.append(total_loss)
    reward_history.append(avg_reward)
    
    if episode % 50 == 0:
        print(f"Episode {episode}: Loss = {total_loss:.2f}, Avg Coverage = {avg_reward*100:.1f}%")

print("Training hoàn tất!")

# Visualize kết quả training
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(reward_history)
plt.title("Coverage Rate (%)")
plt.xlabel("Episode")
plt.subplot(1, 2, 2)
plt.plot(loss_history)
plt.title("Loss")
plt.xlabel("Episode")
plt.show()

Bắt đầu training...


AttributeError: 'tuple' object has no attribute 'size'